In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
cd /content/drive/MyDrive/YR Hacks ML

/content/drive/MyDrive/YR Hacks ML


In [52]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [51]:
import tensorflow
print(tensorflow.__version__)

2.8.0


**Preparing the data sets for training**






In [33]:
import cv2, os
dataSet = "/content/drive/MyDrive/YR Hacks ML/data"
types = os.listdir(dataSet)
typeNames = [i for i in range(len(types))]
labelDict = dict(zip(types,typeNames))

print(labelDict)
print(types)
print(typeNames)

{'melanoma': 0, 'basal cell carcinoma': 1, 'squamous cell carcinoma': 2}
['melanoma', 'basal cell carcinoma', 'squamous cell carcinoma']
[0, 1, 2]


In [34]:
imageSize=100
data=[]
target=[]

for i in types:
  folderPath = os.path.join(dataSet, i)
  imageNames = os.listdir(folderPath)

  for imageName in imageNames:
      imagePath = os.path.join(folderPath,imageName)
      image=cv2.imread(imagePath)
        
      try:  
          resizedImage=cv2.resize(image,(imageSize,imageSize))
          #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
          data.append(resizedImage)
          target.append(labelDict[i])
          #appending the image and the label(categorized) into the list (dataset)
      except Exception as e:
          print('Exception:',e)
          #if any exception rasied, the exception will be printed here. And pass to the next image


**Rescaling/Reshaping training data***

In [36]:
import numpy as np
from keras.utils import np_utils
data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], imageSize,imageSize,3))
target = np.array(target)
updatedTarget = np_utils.to_categorical(target)

In [37]:
updatedTarget.shape #Check the total number of training images, and number of categories

(1043, 3)

**CNN Model***

In [38]:
data.shape #955 images, 100x100, 3 categories

(1043, 100, 100, 3)

In [39]:
data.shape[1:]

(100, 100, 3)

In [40]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 200)       5600      
                                                                 
 activation (Activation)     (None, 98, 98, 200)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 200)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 100)       180100    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 100)      0         
 2D)                                                    

**Splitting into training (70%), testing (10%) and validation (20%)**

In [44]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target = train_test_split(data,updatedTarget,test_size=0.1)

In [45]:
train_data.shape

(938, 100, 100, 3)

In [46]:
train_target.shape

(938, 3)

In [48]:
history=model.fit(train_data,train_target,epochs=200,validation_split=0.2)

Epoch 1/200


ValueError: ignored